# Code chunks for Exercise 2

Welcome to the second Jupyter notebook collecting many code chunks that are useful for the AF VU exercises. In addition to introducing helpful code snippets, we focus on the second exercise this time, i.e. classifying snow melt with radar imagery.

## Some Python libraries you might need

A lot of libraries will be used for the exercise, but don't get intimidated by the amout of imports. They mostly fulfill small tasks and most of them were already covered by the first exercise. 

First, we import some popular, useful libraries.

In [1]:
import earthpy.spatial as es
import datetime
import numpy as np
import xarray as xar
from PIL import Image
from scipy import signal as sig

Then, we continue with in-house production software for all sorts of data processing. For more information have a look at https://github.com/TUW-GEO.

In [2]:
from veranda.io.geotiff import read_tiff
from geopathfinder.folder_naming import build_smarttree
from geopathfinder.naming_conventions.sgrt_naming import SgrtFilename
from yeoda.products.preprocessed import SIG0DataCube

Several tasks cannot be solely managed with the existing general-purpose packages. Therefore, we have created a helper package dedicated for this lecture.

In [3]:
from afe.dc_add_ons import TMENPLIADataCube

Finally, we import `matplotlib` for creating plots and activate its widgets for interactive plots.

In [4]:
from matplotlib import colors
import matplotlib.pyplot as plt
%matplotlib widget  
%matplotlib widget 

To allow for running the code chunks for any user, we can globally define our username here. Please change the user "cnavacch" to your own username.

In [5]:
USER = "e11815736"

## Create hillshade and aspect (dt: Exposition bzw. Hangorientierung) mask from a digital elevation model

First, we need to locate where the digital elevation model (DEM) data is stored and select our file of interest (10000x10000 pixels).

In [6]:
dem_path = f"/home/{USER}/shared/data/auxiliary_data/dem/srtm_vfp/EU010M/E048N015T1/SRTM_VFP_EU010M_E048N015T1.tif"

Then we can load it with `read_tiff` as a `numpy` array. The first return value is the data, the second metadata tags, which we do not need now.

In [ ]:
dem, _ = read_tiff(dem_path)

Then we select elevation data for a specific region of interest.

In [ ]:
elevation = dem[1000:2000, 1000:2000]

Next, we can create a hillshade from the selected elevation data. The `earthpy` library features a function for hillshading that takes elevation data, and an azimuth and altitude value defining the source of illumination. Note that for larger arrays, this operation might take a while. Hillshade is used for visualisation purposes. 

In [ ]:
hillshade1 = es.hillshade(elevation, azimuth=45, altitude=20)

In order to compute the aspect (orientation) of the surfaces, we compute the first derivative in north-south direction, which is realised by convoluting DEM data with a vertical Sobel kernel.

In [ ]:
sob_v = np.array([[1, 2, 1],[0, 0, 0],[-1, -2, -1]], dtype = 'int32')
gradx = sig.convolve2d(elevation, sob_v, mode='same')/8

Finally, we can plot the created hillshade.

In [ ]:
fig = plt.figure(figsize=(11,8))
ax0 = plt.subplot(231)
ax1 = plt.subplot(232)
ax2 = plt.subplot(233)


ax0.set_title('Elevation in m')
el_img = ax0.imshow(elevation, cmap="gist_earth")
fig.colorbar(el_img, ax=ax0, shrink=0.75, orientation='horizontal')
ax1.set_title('Hillshade 45 AZ')
hs_img = ax1.imshow(hillshade1, cmap="Greys_r",vmin=0, vmax=255)
ax2.set_title('Nord-Süd Gradient')
hs_img = ax2.imshow(gradx, vmin=-10, vmax=10)

ax3 = plt.subplot(234)
ax4 = plt.subplot(235)
ax5 = plt.subplot(236)

ax3.set_title('Nordseitige Hänge')
ax3.imshow(gradx > 2)
ax4.set_title('Südseitige Hänge')
ax4.imshow(gradx < -2)
ax5.set_title('Keine NS-Steigung')
hs_img = ax5.imshow(np.abs(gradx) < 2)

## `yeoda` datacubes 

This section deals with the creation, filtering and handling of sigma naught backscatter (SIG0) and projected local incidence angle data (TMENPLIA). `yeoda` offers already datacubes for this kind of tasks, making your life easier.

### Setup

First, we need to define our dimensions of interest. The ones you see below will be relevant for this exercise.

In [ ]:
dimensions = ["pol", "time", "orbit_direction", "relative_orbit", "tile"]

Then, we define the root path to our dataset of interest (replace the user "cnavacch" with your user name). In this case it is the path to the SIG0 data.

In [ ]:
root_dirpath = f"/home/{USER}/shared/data/sentinel1/preprocessed/EU010M"

The `geopathfinder` library can help you to collect all files within the root folder. To do so, you only need to specify the level names below the root folder and a *regex* pattern for selecting the right files. `geopathfinder`'s `build_smarttree()` function then creates a folder tree object for collecting the files in a recursive manner. During Exercise 1, you have already used the `generate_tree()` function, which does the exact same thing, just for a different dataset.

In [ ]:
folder_hierarchy = ["tile", "quantity"]
dir_tree = build_smarttree(root_dirpath, folder_hierarchy, register_file_pattern="^[^Q].*.tif")
filepaths = dir_tree.file_register

Each file name in this dataset follows a certain naming convention, defined and managed by `geopathfinder`'s class `SgrtFilename`. 
The list of file paths can then directly be forwarded to `yeoda`'s `SIG0DataCube`, along with the chosen dimensions, the file naming class, and the data encoding attributes (scale factor, no data value, and data type).

In [ ]:
sig_dc = SIG0DataCube(filepaths=filepaths, dimensions=dimensions, filename_class=SgrtFilename, scale_factor=100, nodata=-9999, dtype='int16')

Now we can take a closer look whats stored in the datacube

In [ ]:
_ = sig_dc.filter_by_dimension('A', name='orbit_direction', inplace=True)

In [ ]:
_ = sig_dc.filter_by_dimension('VV', name='pol', inplace=True)

and only select data in between 2017-12-27 at 6am and 2018-01-01 at 6am.

In [ ]:
_ = sig_dc.filter_by_dimension([(datetime.datetime(2017, 12, 27, 6, 0, 0), datetime.datetime(2018, 1, 1, 6, 0, 0))], expressions=[('>','<=')], name='time', inplace=True)

### Loading data

Note that more than one image can be loaded with a single call of `load_by_pixels` or `load_by_geom`. The resulting array is then 3-dimensional. Be aware that the datacube must only contain image paths from a single tile when using this method, therefore we need to filter it for one tile first. After defining our region of interest

In [ ]:
tilename='E051N016T1'
ul_row = 5800
ul_col = 2860
row_size = 800
col_size = 800

, where `(ul_row, ul_col)` is the upper-left anchor point of the pixel window and `(row_size, col_size)` the shape of the pixel window, we can load the data as a `numpy` array. As there are three image paths in the datacube left, the resulting array has the dimension 3x4000x4000.

In [ ]:
_ = sig_dc.filter_by_dimension(tilename, name='tile_name', inplace=True)
sig0_arr = sig_dc.load_by_pixels(ul_row, ul_col, row_size=row_size, col_size=col_size, dtype='numpy')

### PLIA data

The second important dataset used in this exercise is projected local incidence angle (PLIA) data, which can be managed in the same way as SIG0 backscatter data, but using a different datacube class. The data is now stored at a different location, since the PLIA data you are working with is averaged per orbit.

In [ ]:
root_dirpath = f"/home/{USER}/shared/data/sentinel1/parameters/EU010M"
dir_tree = build_smarttree(root_dirpath, folder_hierarchy, register_file_pattern="^[^Q].*.tif")
filepaths = dir_tree.file_register

For PLIA we need less dimensions than for SIG0.

In [ ]:
dimensions = ["relative_orbit", "tile"]

The initialisation needs also less arguments, since the nodata value, scale factor and data type are already defined within the `TMENPLIADataCube` class. 

In [ ]:
dc = TMENPLIADataCube(filepaths=filepaths, dimensions=dimensions, filename_class=SgrtFilename)

After filtering the datacube by relative orbit

In [ ]:
_ = dc.filter_by_dimension(44, name='relative_orbit', inplace=True)

and tilename

In [ ]:
_ = dc.filter_by_dimension(tilename, name='tile_name', inplace=True)

one file is left in the datacube.

In [ ]:
dc.inventory

Now we can load the data from the same pixel extent.

In [ ]:
plia_arr = dc.load_by_pixels(ul_row, ul_col, row_size=row_size, col_size=col_size, dtype='numpy')
plia_arr

## Displaying images with `matplotlib`

We can also display our SIG0 backscatter data we have retrieved before and plot a colorbar next to it. We can do the same with our loaded PLIA data.

In [ ]:
fig = plt.figure(figsize=(11, 5))
ax1 = plt.subplot(121)
ax1.set_title('Sigma nought backscatter of Enns')
img_h = ax1.imshow(sig0_arr[2], cmap=plt.get_cmap("Greys_r"), vmin=8, vmax=-18)
cb = fig.colorbar(img_h, ax=ax1, shrink=0.75)
cb.set_label("Sigma nought backscatter [dB]")
                 
ax2= plt.subplot(122)
ax2.set_title('PLIA of Enns (orbit 44)')
img_h = ax2.imshow(plia_arr[0], cmap=plt.get_cmap("Greys_r"))#, vmin=20, vmax=70)

cb = fig.colorbar(img_h, ax=ax2, shrink=0.75)
cb.set_label("PLIA [°]")                 

## Creating custom colormaps

In Exercise 2 you will need to work with more than two classes, thus a custom colormap can be helpful to visualise and analyse your classification outcome. `matplotlib`'s `ListedColormap` allows you to create a colormap from a list containing `n` colors.

**Hint:** Using RGBA values for colors, some pixels can be set to be fully transparent. This can be useful if you want to overlay two images. You might also want to take a look at the `set_under` and `set_over` methods of colormaps.

In [ ]:
cmap = colors.ListedColormap(['red', 'blue', 'turquoise', 'white'])

The value ranges referring to each of the colors must be specified in a second array with a length of `n+1` defining the upper and lower value boundary allocated to a certain color.

In [ ]:
bounds = np.array([-1000, -12, -8, -6, 0])

The number of elements in the colormap `cmap.N` and the class boundaries can then be used to create an object for normalising the values to 0 and 1. 

In [ ]:
norm = colors.BoundaryNorm(bounds, cmap.N)

Now we can plot the same image as before, but with the discrete colormap of our choice.

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot()
img_col = ax.imshow(sig0_arr[0], cmap=cmap, norm=norm)

cbar = plt.colorbar(img_col, ax=ax, boundaries=bounds, ticks=bounds[1:], shrink=0.65, orientation='horizontal')
cbar.set_label('Sigma nought backscatter [dB]')